In [3]:
!pip install ultralytics
!pip install opencv-python-headless


Defaulting to user installation because normal site-packages is not writeable


DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\certifi-2024.12.14-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\charset_normalizer-3.4.1-py3.11-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\idna-3.10-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\p

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python_headless-4.11.0.86-cp37-abi3-win_amd64.whl (39.4 MB)


DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\certifi-2024.12.14-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\charset_normalizer-3.4.1-py3.11-win-amd64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\python\python311\site-packages\idna-3.10-py3.11.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\stakh\appdata\roaming\p

In [2]:
import os
import shutil
import random

# Исходные пути:
# Пусть изображения лежат по классам в папке source_images_path
# И YOLO-метки (.txt) созданы в параллельной структуре в source_labels_path
source_images_path = "dataset-resized"  # Например: /mnt/data/dataset-resized/plastic, /mnt/data/dataset-resized/metal, ...
source_labels_path = "dataset-resized/yolo_labels"        # Аналогичная структура с метками: /mnt/data/yolo_labels/plastic, /mnt/data/yolo_labels/metal, ...

# Путь к новой структуре, которая будет использоваться для тренировки
destination_base = "dataset-resized/dataset_yolo"
train_images_dir = os.path.join(destination_base, "images", "train")
train_labels_dir = os.path.join(destination_base, "labels", "train")
val_images_dir   = os.path.join(destination_base, "images", "val")
val_labels_dir   = os.path.join(destination_base, "labels", "val")

# Создаем необходимые папки
for d in [train_images_dir, train_labels_dir, val_images_dir, val_labels_dir]:
    os.makedirs(d, exist_ok=True)

# Доля изображений, которые уйдут в валидацию (например, 20%)
val_ratio = 0.2

# Получаем список классов (имена подпапок в исходном датасете)
class_names = sorted([item for item in os.listdir(source_images_path)
                      if os.path.isdir(os.path.join(source_images_path, item))])

# Для каждого класса копируем изображения и соответствующие .txt файлы в train/val с новым именованием, чтобы избежать конфликтов
for class_name in class_names:
    class_images_dir = os.path.join(source_images_path, class_name)
    class_labels_dir = os.path.join(source_labels_path, class_name)
    
    # Список изображений (расширения .jpg, .jpeg, .png)
    image_files = [f for f in os.listdir(class_images_dir) if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    
    # Перемешиваем для случайного разделения
    random.shuffle(image_files)
    
    num_val = int(len(image_files) * val_ratio)
    val_files = image_files[:num_val]
    train_files = image_files[num_val:]
    
    for f in train_files:
        # Новое имя файла: добавляем префикс с именем класса
        new_filename = f"{class_name}_{f}"
        # Копируем изображение
        src_img = os.path.join(class_images_dir, f)
        dst_img = os.path.join(train_images_dir, new_filename)
        shutil.copy(src_img, dst_img)
        
        # Копируем соответствующий .txt файл (если он есть)
        src_label = os.path.join(class_labels_dir, os.path.splitext(f)[0] + ".txt")
        dst_label = os.path.join(train_labels_dir, os.path.splitext(new_filename)[0] + ".txt")
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)
    
    for f in val_files:
        new_filename = f"{class_name}_{f}"
        src_img = os.path.join(class_images_dir, f)
        dst_img = os.path.join(val_images_dir, new_filename)
        shutil.copy(src_img, dst_img)
        
        src_label = os.path.join(class_labels_dir, os.path.splitext(f)[0] + ".txt")
        dst_label = os.path.join(val_labels_dir, os.path.splitext(new_filename)[0] + ".txt")
        if os.path.exists(src_label):
            shutil.copy(src_label, dst_label)

print(f"✅ Изображения и метки успешно разделены по train и val.")

# Создаем файл data.yaml для тренировки YOLO (например, для YOLOv8)
data_yaml = f"""\
train: {train_images_dir}
val: {val_images_dir}

nc: {len(class_names)}
names: {class_names}
"""

data_yaml_path = os.path.join(destination_base, "data.yaml")
with open(data_yaml_path, "w", encoding="utf-8") as f:
    f.write(data_yaml)

print(f"✅ Файл data.yaml создан: {data_yaml_path}")


KeyboardInterrupt: 

In [4]:
import os
from ultralytics import YOLO

# Пути из вашей разметки
dataset_path = "dataset-resized"
yolo_labels_path = "dataset-resized/yolo_labels"

# Проверяем файлы разметки
os.makedirs(dataset_path, exist_ok=True)
os.makedirs(yolo_labels_path, exist_ok=True)

print("Файлы в датасете:", os.listdir(dataset_path))
print("Файлы в разметке YOLO:", os.listdir(yolo_labels_path)[:10])  # Выведем первые 10


Файлы в датасете: ['annotations.json', 'cardboard', 'dataset_yolo', 'glass', 'metal', 'paper', 'plastic', 'trash', 'yolo_labels']
Файлы в разметке YOLO: ['cardboard', 'glass', 'metal', 'paper', 'plastic', 'trash']


In [5]:
import os

dataset_path = "dataset-resized"

# Проверяем содержимое датасета
print("Содержимое dataset-resized:", os.listdir(dataset_path))


Содержимое dataset-resized: ['annotations.json', 'cardboard', 'dataset_yolo', 'glass', 'metal', 'paper', 'plastic', 'trash', 'yolo_labels']


In [6]:
import torch

print(f"✅ CUDA доступен: {torch.cuda.is_available()}")
print(f"🖥 Количество GPU: {torch.cuda.device_count()}")
print(f"🔹 Доступное устройство: {torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'Нет GPU'}")


✅ CUDA доступен: True
🖥 Количество GPU: 1
🔹 Доступное устройство: NVIDIA GeForce GTX 1650


In [7]:
from ultralytics import YOLO

# Загружаем модель
model = YOLO("yolov8n.pt")  # Можно попробовать yolov8m.pt

# Обучаем с увеличенным числом эпох, аугментацией и сниженным learning rate
model.train(
    data="dataset-resized/dataset_yolo/data.yaml",
    epochs=150,
    batch=16,
    imgsz=512,
    device="cuda",
    lr0=0.001,      # Сниженный learning rate
    augment=True    # Включаем аугментацию данных
)

New https://pypi.org/project/ultralytics/8.3.72 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=dataset-resized/dataset_yolo/data.yaml, epochs=150, time=None, patience=100, batch=16, imgsz=512, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=None, name=train10, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=True, agnostic_nms=False, classes=None, retina_masks=False, embed=None,

train: Scanning C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\dataset_yolo\labels\train.cache... 2143 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2143/2143 [00:00<?, ?it/s]
val: Scanning C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\dataset_yolo\labels\val.cache... 563 images, 0 backgrounds, 0 corrupt: 100%|██████████| 563/563 [00:00<?, ?it/s]


Plotting labels to runs\detect\train10\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.001' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 512 train, 512 val
Using 8 dataloader workers
Logging results to runs\detect\train10
Starting training for 150 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      1/150      2.62G     0.2218      2.205     0.9701         47        512: 100%|██████████| 134/134 [00:37<00:00,  3.58it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:04<00:00,  4.07it/s]


                   all        563        563      0.696      0.542      0.603      0.586

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        563        563      0.931      0.972       0.98       0.98



150 epochs completed in 1.683 hours.
Optimizer stripped from runs\detect\train10\weights\last.pt, 6.2MB
Optimizer stripped from runs\detect\train10\weights\best.pt, 6.2MB

Validating runs\detect\train10\weights\best.pt...
Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:07<00:00,  2.57it/s]


                   all        563        563      0.962      0.942       0.99       0.99
             cardboard        140        140      0.972      0.986      0.995      0.995
                 glass        100        100      0.971      0.988      0.993      0.993
                 metal         82         82      0.982      0.915      0.992      0.992
                 paper        118        118      0.995      0.924       0.99       0.99
               plastic         96         96      0.988      0.877      0.991      0.991
                 trash         27         27      0.865      0.963       0.98       0.98
Speed: 0.2ms preprocess, 7.8ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs\detect\train10


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2, 3, 4, 5])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002303C07A150>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
 

In [8]:
from ultralytics import YOLO

# Загружаем обученную модель
best_model_path = "runs/detect/train10/weights/best.pt"
model = YOLO(best_model_path)

# Валидация модели
metrics = model.val()

# Вывод метрик
print("📊 Итоговые метрики после обучения:")
print(f"🔹 mAP@50: {metrics.box.map:.4f}")
print(f"🔹 mAP@50-95: {metrics.box.map50:.4f}")
print(f"🔹 Precision (Точность): {metrics.box.mp:.4f}")
print(f"🔹 Recall (Полнота): {metrics.box.mr:.4f}")


Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3,006,818 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\dataset_yolo\labels\val.cache... 563 images, 0 backgrounds, 0 corrupt: 100%|██████████| 563/563 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:04<00:00,  7.88it/s]


                   all        563        563      0.937      0.944      0.985      0.985
             cardboard        140        140      0.963      0.986      0.994      0.994
                 glass        100        100      0.972       0.98      0.993      0.993
                 metal         82         82      0.975      0.958      0.992      0.992
                 paper        118        118      0.972       0.89      0.984      0.983
               plastic         96         96      0.977      0.889      0.988      0.988
                 trash         27         27      0.762      0.963      0.962      0.962
Speed: 0.1ms preprocess, 3.3ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\val3
📊 Итоговые метрики после обучения:
🔹 mAP@50: 0.9852
🔹 mAP@50-95: 0.9854
🔹 Precision (Точность): 0.9369
🔹 Recall (Полнота): 0.9443


In [1]:
from ultralytics import YOLO

# Загружаем обученную модель
model = YOLO("runs/detect/train9/weights/best.pt")
# Экспорт в формат TorchScript
model.export(format="torchscript")


Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CPU (Intel Core(TM) i5-10400F 2.90GHz)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs\detect\train9\weights\best.pt' with input shape (1, 3, 416, 416) BCHW and output shape(s) (1, 11, 3549) (5.9 MB)

TorchScript: starting export with torch 2.5.1+cu121...
TorchScript: export success  1.8s, saved as 'runs\detect\train9\weights\best.torchscript' (11.8 MB)

Export complete (11.0s)
Results saved to C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\runs\detect\train9\weights
Predict:         yolo predict task=detect model=runs\detect\train9\weights\best.torchscript imgsz=416  
Validate:        yolo val task=detect model=runs\detect\train9\weights\best.torchscript imgsz=416 data=dataset-resized/dataset_yolo/data.yaml  
Visualize:       https://netron.app


'runs\\detect\\train9\\weights\\best.torchscript'

In [2]:
from ultralytics import YOLO

# Load a model
model = YOLO("runs/detect/train9/weights/best.pt")

# Validate the model
metrics = model.val()  # no arguments needed, dataset and settings remembered
metrics.box.map    # map50-95
metrics.box.map50  # map50
metrics.box.map75  # map75
metrics.box.maps   # a list contains map50-95 of each category
# Загружаем обученную модель



Ultralytics 8.3.71  Python-3.11.0 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
Model summary (fused): 168 layers, 3,007,013 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning C:\Users\stakh\Downloads\Rubbish-20250205T162630Z-001\Rubbish\dataset-resized\dataset_yolo\labels\val.cache... 563 images, 0 backgrounds, 0 corrupt: 100%|██████████| 563/563 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 36/36 [00:04<00:00,  8.69it/s]


                   all        563        563      0.952      0.943      0.987      0.987
             cardboard        140        140      0.993      0.979      0.995      0.995
          dataset_yolo        100        100      0.952          1      0.994      0.994
                 glass         82         82      0.987      0.926      0.983      0.983
                 metal        118        118      0.971      0.856      0.983      0.981
                 paper         96         96      0.977      0.896      0.986      0.986
               plastic         27         27      0.832          1      0.983      0.983
Speed: 0.2ms preprocess, 2.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val2


array([    0.99465,     0.99421,     0.98321,     0.98126,     0.98624,     0.98321,     0.98713])